In [2]:
import os, json
import pandas as pd
import numpy as np
import glob
import re
import tqdm as tqdm
import ast
import gc
import resource

In [3]:
# from sql_loader import SqlLoader
def limit_memory(maxsize):
    # Set the maximum memory limit (in bytes)
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))

In [4]:
limit_memory(12 * 1024 * 1024 * 1024)  # Limit to 8 GB 


In [5]:
'''   

input: json_folder
- linearly iterate the fodler and run single file records.
- exports single to another CSV folder (output)
- 
output:


'''
def run_all_json():
    # for json file in folder
    #   run read_single_file json
    # # export_single_file to single file csv to csv fodler
    # return 0
    pass 

In [6]:
'''
optimize memory: 
input: single linear loop of json logs.
read / write datafame of single file
output: single file linear iterate and write to csv.
''' 

def read_single_file(json_file): # call at bottom
    # read json (json_path)
    df = pd.read_json(json_file) # read data frame from json file
    # reformat: 
    result_column_df = df['data']['result']
    all_values = []
    for i in range(len(result_column_df)): 
        values = df['data']['result'][i]['values']  
        all_values.extend(values)  
    values_df = pd.Series(all_values)
    values_df_rev = values_df.iloc[::-1]
    return values_df_rev


In [7]:
'''    
get & fetch single file
read single file.
'''
def collect_all_events(series):
    '''  
    Input: series variable of all read logs.
    Output: a dataframe with events as data points in string format.
    '''
    start_indices = []
    end_indices = []
    for i, value in enumerate((series)):
        if 'Start fetch event' in str(value):
            start_indices.append(i)
        elif '======END======' in str(value):
            end_indices.append(i)  
    log_segments = []
    end_index = 0 

    for start in (start_indices):
        while end_index < len(end_indices) and end_indices[end_index] <= start:
            end_index += 1
        if end_index < len(end_indices):
            end = end_indices[end_index]
            segment = ' '.join(str(series[i]) for i in range(start + 1, end))
            log_segments.append(segment)
    event_data_df = pd.DataFrame(log_segments, columns=['Log String'])
    event_data_df.index.name = 'Index'  
    event_all_df = event_data_df['Log String']
    return event_all_df  






In [8]:
def get_args(log):
    '''  
    input: log strings in JSON
    output: arg data fields as dataFrame for single data point
    '''
    
    if 'arg: {' in log:
        arg_match = re.search(r'arg:\s*(\{.*?\})', log)
        if arg_match:
            args_string = arg_match.group(1)
            args_dict = ast.literal_eval(args_string) ##
            df = pd.json_normalize(args_dict)
            return df

In [9]:
def fetch_args(event_all_df):
        '''  
        input: 
        output: dataFrame of all event args
        '''
        args_dfs = []
        for log in ((event_all_df)):
            log = str(log)
            if isinstance(log, list):
                log = log[0]
            args_df = get_args(log)
            args_dfs.append(args_df) # Null error
        final_args_df = pd.concat(args_dfs, ignore_index=True)
        
        return(final_args_df)

In [10]:
def get_request_event_list(log):
        '''  
        '''
        if 'Request get event list' in log:
            request_match = re.search(r'Request get event list: (.*?)}', log) 
            if request_match:
                request_string = request_match.group(1) + '}' 
                request_dict = ast.literal_eval(request_string)
                df = pd.json_normalize(request_dict)
                return df
                
            

    
def fetch_request_event_list(event_all_df):
    ''' 
    
    '''
    request_event_list_dfs = []
    for log in ((event_all_df)):
        if isinstance(log, list):
            log = log[0]  
        req_df = get_request_event_list(log)
        request_event_list_dfs.append(req_df)
    final_request_event_list_df = pd.concat(request_event_list_dfs, ignore_index=True)
    return(final_request_event_list_df)

In [11]:
# get and fetch all, gather into single csv. (run_single_json)
# loop the folder (run_json_fodler) ===> csv folder as ouut. 


def get_delay_alarm(log):
    ''' 
    
    '''
    if 'Delay alarm event: ' in log: 
        delay_alarm_event_match = re.search(r'Delay alarm event: (.*?) s', log) 
        if delay_alarm_event_match:
            delay_alarm_event_string = delay_alarm_event_match.group(1) 
            return pd.DataFrame({'DelayAlarm': [delay_alarm_event_string]})
        
    


    
def fetch_delay_alarm(event_all_df):
    ''' 
    
    '''
    delay_alarm_dfs = []
    for log in ((event_all_df)):
        if isinstance(log, list):
            log = log[0] 
        delay_alarm_event_df = get_delay_alarm(log)
        delay_alarm_dfs.append(delay_alarm_event_df) 
    final_delay_alarm_df = pd.concat(delay_alarm_dfs, ignore_index=True)
    return final_delay_alarm_df



In [12]:

def get_time_query_event(log):
    ''' 
    
    '''
    if 'Time query event: ' in log:
        time_query_event_match = re.search(r'Time query event: (\d+\.\d{3})', log) # search string & end string 
        if time_query_event_match:
            time_query_event_string = time_query_event_match.group(1) 
            return pd.DataFrame({'TimeQueryEvent': [time_query_event_string]})
        else:
            return pd.DataFrame({'TimeQueryEvent': ['Not Found'],
                    })
    else:
        return pd.DataFrame({'TimeQueryEvent': ['Not Found'],
                    })

def fetch_time_query_event(event_all_df):
    ''' 
    
    '''
    time_query_event_dfs = []
    for log in ((event_all_df)):
        if isinstance(log, list):
            log = log[0]  
        time_query_event_df = get_time_query_event(log)
        time_query_event_dfs.append(time_query_event_df) 
    final_time_query_event_df = pd.concat(time_query_event_dfs, ignore_index=True)
    return(final_time_query_event_df)

In [13]:
def get_time_process_home_face(log):
        '''
         
        '''
        if 'Time query event: ' in log:
            time_time_process_home_face_match = re.search(r'Time query event: (\d+\.\d{3})', log) 
            if time_time_process_home_face_match:
                time_time_process_home_face_string = time_time_process_home_face_match.group(1)
                return pd.DataFrame({'TimeHomeFace': [time_time_process_home_face_string]})
            else:
                return pd.DataFrame({'TimeHomeFace': ['Not Found'],
                                    })
        else:
            return pd.DataFrame({'TimeHomeFace': ['Not Found'],
                        })
        
def fetch_time_process_home_face(event_all_df):
    '''  
    '''
    time_process_home_face_dfs = []
    for log in ((event_all_df)):
        if isinstance(log, list):
            log = log[0]  
        time_process_home_face_df = get_time_process_home_face(log)
        time_process_home_face_dfs.append(time_process_home_face_df) 
    final_time_process_home_face_df = pd.concat(time_process_home_face_dfs, ignore_index=True)
    return final_time_process_home_face_df #df

In [14]:
def get_all_time_worker(log):
        ''' 
        
        
        '''
        if 'All time worker' in log:
            all_time_worker_match = re.search(r"All time worker: (.*?) s" , log)
            if all_time_worker_match:
                all_time_worker_string = all_time_worker_match.group(1) 
                df = pd.DataFrame({'AllTimeWorker': [all_time_worker_string]})
                return df
            else:
                pass
                return pd.DataFrame({
                        'AllTimeWorker': ['Not Found'],
                        })    
        else:
            pass
            return pd.DataFrame({
                        'AllTimeWorker': ['Not Found'],
                        })
        
        
    

def fetch_all_time_worker(event_all_df):
    ''' 
    
    '''
    all_time_worker_dfs = []
    for log in ((event_all_df)):
        if isinstance(log, list):
            log = log[0]  

        all_time_worker_event_df = get_all_time_worker(log)
        all_time_worker_dfs.append(all_time_worker_event_df) 
    final_all_time_worker_df = pd.concat(all_time_worker_dfs, ignore_index=True)
    return final_all_time_worker_df

In [15]:
def get_response_alarm(log):
        if 'Response get event list: ' in log: 
                response_match = re.search(r" 'data': (.*?)'IsFinished': '1'}}" , log)
                if response_match:
                    response_string = response_match.group(1) + '}' 
                    response_string = response_string.replace("'", '"')
                    response_string = re.sub(r'"recface":\s*"({.*?})"', r'"recface": \1', response_string)
                    response_string = re.sub(r',\s*([\]})])', r'\1', response_string)
                    response_dict = ast.literal_eval(response_string)
                    df = pd.json_normalize(
                        response_dict, 
                        record_path=  ['AlarmArray'],
                        meta = ['SerialNumber'],
                    )
                    return df
        

def fetch_response_alarm(event_all_df):
    response_event_dfs = []
    for log in ((event_all_df)):
        if isinstance(log, list):
            log = log[0] 
        response_event_df = get_response_alarm(log)
        response_event_dfs.append(response_event_df)
    final_response_event_df = pd.concat(response_event_dfs, ignore_index=True)
    return final_response_event_df

In [16]:
def fetch_all(event_all_df):
    ''' 
    '''
    df_result = pd.concat(
                    [
                    fetch_args(event_all_df),
                    fetch_request_event_list(event_all_df),
                    fetch_time_query_event(event_all_df),
                    fetch_time_process_home_face(event_all_df),
                    fetch_all_time_worker(event_all_df)
                    ],
                    # ignore_index=True,
                    axis=1)
    response_alarm = fetch_response_alarm(event_all_df)
    delay_alarm = fetch_delay_alarm(event_all_df)       
    n = len(response_alarm)
    response_alarm_subset = response_alarm.head(n)  
    delay_alarm_subset = delay_alarm.head(n)    
    
    alarm_df = pd.concat(
        [response_alarm_subset, delay_alarm_subset],
        axis=1
    )
    alarm_df.drop('AlarmMsg', axis = 1)    
    # alarm_df = pd.concat(
    #                 [
    #                 .fetch_response_alarm(),
    #                 .fetch_delay_alarm()
    #                 ],
    #                 # ignore_index=True,
    #                 axis=1)
    
    # postprocessing
    
    df_result = df_result.drop('token',axis = 1)
    df_result = df_result.drop('time_millis',axis = 1)
    df_result = df_result.drop('encrypted_str',axis = 1)
    df_result = df_result.drop('startTime',axis = 1)
    df_result = df_result.drop('endTime',axis = 1)
    df_result = df_result.replace('Not Found', pd.NA).dropna() # not NA

    return df_result , alarm_df

In [17]:
def write_to_csv_folder(file_index,
                        event_df,
                        alarm_df,
                        event_csv_path,
                        alarm_csv_path,
                        chunk_size=10):  # Adjust chunk size as needed
    """
    Write two DataFrames to specified CSV file paths in chunks.
    Parameters:
    - file_index (int): The index used to create the CSV file names.
    - event_df (pd.DataFrame): DataFrame containing event data.
    - alarm_df (pd.DataFrame): DataFrame containing alarm data.
    - event_csv_path (str): Directory path to save the event CSV file.
    - alarm_csv_path (str): Directory path to save the alarm CSV file.
    - chunk_size (int): Number of rows to write at a time.
    """
    event_file_name = f"event_data_{file_index}.csv"
    alarm_file_name = f"alarm_data_{file_index}.csv"
    
    event_file_path = os.path.join(event_csv_path, event_file_name)
    alarm_file_path = os.path.join(alarm_csv_path, alarm_file_name)
    
    # Write event_df to CSV in chunks
    for i in range(0, len(event_df), chunk_size):
        event_df.iloc[i:i + chunk_size].to_csv(event_file_path, mode='a', header=(i == 0), index=False)

    # Write alarm_df to CSV in chunks
    for i in range(0, len(alarm_df), chunk_size):
        alarm_df.iloc[i:i + chunk_size].to_csv(alarm_file_path, mode='a', header=(i == 0), index=False)

    # Free memory by deleting DataFrames
    del event_df
    del alarm_df

In [18]:

def write_all(json_folder_path):
# iteratively read json files.
    index = 0
    for filename in tqdm.tqdm(os.listdir(json_folder_path), desc ="Reading json files: "):
        if filename.endswith('.json'):  # Ensure the file is a JSON file 
            file_path = os.path.join(json_folder_path, filename)
            json_df = read_single_file(file_path)
            event_all_df = collect_all_events(json_df)
            event_df , alarm_df = fetch_all(event_all_df)
            
            # run sql upload (packaged)
             
            # write_to_csv_folder(index,
            # event_df,
            # alarm_df, 
            # 'csv_output/event_csv/',
            # 'csv_output/alarm_csv/')
        index += 1
    pass






In [19]:
write_all('json_logs')
# kernel crash at 15/18 
# memory error

Reading json files: 100%|██████████| 18/18 [09:52<00:00, 32.91s/it]


In [20]:
#todo: modify code log script & read from csv to SQL upload.